<a href="https://colab.research.google.com/github/Megaexoplanet/project-3/blob/master/PThree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Basic libaries
import numpy as np
from datetime import datetime as dt, timedelta as td
# Data manipulation
import pandas as pd
import csv
# Modelling
import statsmodels.api as sm
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
import scipy as sp
import tensorflow as tf
# Visualizations
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
matplotlib.use('SVG')

In [ ]:
# Changing Frequency
freq=td(hours=1)
# Choose missing_data_handling to be True to fill missing data points with a concave function and False to remove missing data points.
missing_data_handling=False

In [ ]:
# reading in stock index date
prices=pd.read_csv("prices_60m.csv",parse_dates=['Dates'])
prices.rename(columns={'Dates':'dates'},inplace=True)
prices=prices.set_index('dates')
prices=prices.dropna(how='any')
prices

if missing_data_handling:
    prices=prices.resample(freq,base=15.5).asfreq()
    
# calculating logreturns and Z Scores
def logreturn(x):
    return np.log(x/x.shift(1))

def zscore(x):
    return (x-x.mean())/x.std()

prices['DJIA LogRet']=logreturn(prices['DJIA CLOSE'])
prices['SP500 LogRet']=logreturn(prices['SP500 CLOSE'])
prices['DJIA Z Scores']=zscore(prices['DJIA CLOSE'])
prices['SP500 Z Scores']=zscore(prices['SP500 CLOSE'])
prices['DJIA Log']=np.log(prices['DJIA CLOSE'])
prices['SP500 Log']=np.log(prices['SP500 CLOSE'])
prices=prices.dropna(how='any')
prices

,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log
dates,,,,,,,,
2019-06-21 17:30:00,26791.51,2952.75,-0.001901,-0.001739,0.241810,-0.054862,10.195840,7.990492
2019-06-21 18:30:00,26796.05,2953.07,0.000169,0.000108,0.250797,-0.048422,10.196010,7.990601
2019-06-21 19:30:00,26790.67,2953.51,-0.000201,0.000149,0.240148,-0.039568,10.195809,7.990750
2019-06-21 20:30:00,26793.13,2955.41,0.000092,0.000643,0.245017,-0.001331,10.195901,7.991393
2019-06-21 21:30:00,26719.13,2950.46,-0.002766,-0.001676,0.098545,-0.100948,10.193135,7.989716
...,...,...,...,...,...,...,...,...
2019-09-27 17:30:00,26890.71,2970.09,-0.004136,-0.004612,0.438163,0.294099,10.199536,7.996348
2019-09-27 18:30:00,26825.86,2961.04,-0.002415,-0.003052,0.309801,0.111971,10.197122,7.993296
2019-09-27 19:30:00,26776.07,2954.41,-0.001858,-0.002242,0.211249,-0.021455,10.195264,7.991054


In [ ]:
# Reading in csv file containing tweets and converting it to a pandas dataframe
tweets=pd.read_csv("raw_tweets.csv",parse_dates=['created_at'])
tweets = tweets.drop(columns="id")

# Converting time from UTC to CEST +2 hours
cest_date=[]
for date in tweets['created_at']:
    cest_date.append(date+td(hours=2))
tweets['created_at']=cest_date

# Changing index to Datetimeindex
tweets.rename(columns={'created_at':'date'},inplace=True)
tweets=tweets.set_index('date')
#tweets.index = pd.DatetimeIndex(tweets.index)

tweets

,tweet_text,bullish
date,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession ri...,NaN
2019-06-24 19:58:00,@nyika_yedu The multi currency regime dealt wi...,NaN
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying ...,NaN
2019-06-24 19:58:00,"The caution continues. ""Small business owners ...",NaN
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - htt...,NaN
...,...,...
2019-09-03 20:47:00,@AmandaAgatiPNC @NFIB Are you a recession deni...,NaN
2019-09-03 20:47:00,NaN,True
2019-09-03 20:47:00,@pdtugs @CPC_HQ Actually Canada economy is goo...,NaN


In [ ]:
len(tweets)

1048575

In [ ]:
tweets['Time']=tweets.index.hour

In [ ]:
df=tweets.groupby(tweets.index.date).count()
df['Total']=df['tweet_text']+df['bullish']
df['Total'].mean()

14563.541666666666

In [ ]:
#tweets['Time'].value_counts().sort_index().plot.bar()
fig, ax = plt.subplots()
counts, bins, patches = ax.hist(tweets.Time, facecolor='gray', edgecolor='black', bins=range(0,25))
ax.set_xticks(bins)
ax.set_title('Tweetek óránkénti eloszlása', fontsize=9)
ax.tick_params(axis='both', which='major', labelsize=7)
plt.savefig("hourly_tweet_distribution.svg", format="svg")
print(counts)
print(bins)

[43588. 38517. 37106. 34865. 33845. 31385. 28502. 25502. 28511. 31300.
 29118. 29834. 36889. 41597. 55131. 61533. 63697. 62545. 61488. 56832.
 55641. 54242. 57459. 49448.]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]


In [ ]:
if not missing_data_handling:
    # Removing tweets that were created on weekends and bank holidays
    #getting trading dates from DJIA intraday prices
    t_dates=prices.index.map(pd.Timestamp.date).unique()
    str_t_dates=[str(x) for x in t_dates]
    tweets=tweets[tweets.index.floor('D').isin(str_t_dates)]
    # Removing tweets that are not in trading analysis interval
    start=str(td(hours=15,minutes=30)-freq)
    end=str(td(hours=21,minutes=29))
    tweets = tweets.between_time(start, end, include_start=True, include_end=True)

In [ ]:
# Separating sentiment tweets from bullish/bearish index tweets
text_tweets=tweets.loc[:, ['tweet_text']]
market_tweets=tweets.loc[:, ['bullish']]

In [ ]:
# Filtering out NaNs
fmarket_tweets=market_tweets[market_tweets['bullish'].notnull()]
ftext_tweets=text_tweets[text_tweets['tweet_text'].notnull()]
ftext_tweets =ftext_tweets.drop_duplicates(subset="tweet_text",keep=False)
print('Total number of observations including both text and bullish/bearish tweets:',ftext_tweets.tweet_text.count()+fmarket_tweets.bullish.count())
ftext_tweets

Total number of observations including both text and bullish/bearish tweets: 327195


,tweet_text
date,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession ri...
2019-06-24 19:58:00,@nyika_yedu The multi currency regime dealt wi...
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying ...
2019-06-24 19:58:00,"The caution continues. ""Small business owners ..."
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - htt...
...,...
2019-09-03 20:47:00,"@PrimeXbt In general, the SP500 is clearly ove..."
2019-09-03 20:47:00,@AmandaAgatiPNC @NFIB Are you a recession deni...
2019-09-03 20:47:00,@pdtugs @CPC_HQ Actually Canada economy is goo...


In [ ]:
text = " ".join(tweet for tweet in ftext_tweets.tweet_text)
print ('There are %s words in the combination of all review.' % (len(text)))

There are 49648007 words in the combination of all review.


In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords_raw=["one","co","amp","https","bullish", "bearish", "stock market", "I think economy", "I feel economy", "I am feeling economy", "It feels economy","inflation", "unemployment rate", "recession","SP500","S&P500","DJIA","Dow jones"]
stopwords_splited=[]
for phrase in stopwords_raw:
    split=phrase.split()
    stopwords_splited=list(set(stopwords_splited+split))
stopwords.update(stopwords_splited)

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white",width=800, height=400).generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
fig = plt.gcf() #get current figure
#fig.set_size_inches(10,10)
#plt.savefig("tweets_word_cloud.png", format="png")

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Matplotlib is currently using svg, which is a non-GUI backend, so cannot show the figure.
  app.launch_new_instance()


In [ ]:
pd.set_option('display.max_colwidth', -1)
ftext_tweets[ftext_tweets['tweet_text'].str.contains("now")].sample(10)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,tweet_text
date,
2019-07-05 18:30:00,"@Mikeystaj @hmacbe I know, I know...more defaults would mean a huge downturn in the economy, as people become bankrupt...but as a first time buyer who has been shafted out of the market, I can't help but feel at least a little bit upset, lol."
2019-07-19 19:44:00,@lovesbasesDFS I can definitely use the stock market TO gamble. Many good traders go on tilt and make knowingly bad trades as a result.
2019-08-14 19:17:00,@realDonaldTrump @cvpayne @Varneyco good to know that our president's solution to the recession he's creating is to deny that it's happening and do nothing. what could possibly go wrong
2019-08-14 20:20:00,"@PeoplesWorld Nah. I don’t think they’re apologizing for colonialism they are just reasserting their desire to maintain independence from mainland China and preserve actual democracy and free market economy. They know the history, that’s exactly why this is such a powerful symbolic act."
2019-08-21 14:42:00,"@realDonaldTrump ""The Fake News LameStream Media"" know they would bankrupt themselves by creating a recession. \n\nThey would NEVER EVER do this.\n\n#PresidentPinocchio"
2019-08-01 19:40:00,@realDonaldTrump Your an idiot and now there will be a massive recession con man. China lays a 100 year game. They can wait.
2019-08-15 18:21:00,@realDonaldTrump Perfect timing for a recession after you cut corporate tax and bailed out Wall Street. Now regular people can suffer as their retirement and 401K tanks but the brightside is that big corporations now have money to buy on the dip of the stock market. #TrumpSlump
2019-07-31 16:35:00,"@brianklaas The #Australian economy has been growing for 27 years, a result of economic reforms 25-35 years ago, &amp; proximity to growing Asian markets. The stock market hit a record just now. No one cared &amp; the PM didn’t “claim it” as “his”. Would be weird if he did. https://t.co/P4Gj4LgFXM"
2019-08-22 19:49:00,@sam__iammm Recession fear put gold on a run. But I didn’t know Target put out a strong Q2 earnings and the report dropped yesterday so ppl along with the Federal Reserve publishing it’s July meeting. So ppl slowed on gold and returned to stocks.


In [ ]:
# import nltk; nltk.download('popular')
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia=SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
del sia.lexicon['thank']
del sia.lexicon['thanks']

In [ ]:
stock_lex = pd.read_csv('stock_lex.csv')
stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
#filtering out multiple words expressions from the lexicon
stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
stock_lex_scaled = {}
for k, v in stock_lex.items():
    if v > 0:
        stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
    else:
        stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

In [ ]:
positive = []
with open('lm_positive.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        positive.append(row[0].strip())
        
negative = []
with open('lm_negative.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        entry = row[0].strip().split(" ")
        if len(entry) > 1:
            negative.extend(entry)
        else:
            negative.append(entry[0])

In [ ]:
final_lex = {}
final_lex.update({word:2.0 for word in positive})
final_lex.update({word:-2.0 for word in negative})
final_lex.update(stock_lex_scaled)
final_lex.update(sia.lexicon)
sia.lexicon = final_lex

In [ ]:
final_lex

{'ABLE': 2.0,
 'ABUNDANCE': 2.0,
 'ABUNDANT': 2.0,
 'ACCLAIMED': 2.0,
 'ACCOMPLISH': 2.0,
 'ACCOMPLISHED': 2.0,
 'ACCOMPLISHES': 2.0,
 'ACCOMPLISHING': 2.0,
 'ACCOMPLISHMENT': 2.0,
 'ACCOMPLISHMENTS': 2.0,
 'ACHIEVE': 2.0,
 'ACHIEVED': 2.0,
 'ACHIEVEMENT': 2.0,
 'ACHIEVEMENTS': 2.0,
 'ACHIEVES': 2.0,
 'ACHIEVING': 2.0,
 'ADEQUATELY': 2.0,
 'ADVANCEMENT': 2.0,
 'ADVANCEMENTS': 2.0,
 'ADVANCES': 2.0,
 'ADVANCING': 2.0,
 'ADVANTAGE': 2.0,
 'ADVANTAGED': 2.0,
 'ADVANTAGEOUS': 2.0,
 'ADVANTAGEOUSLY': 2.0,
 'ADVANTAGES': 2.0,
 'ALLIANCE': 2.0,
 'ALLIANCES': 2.0,
 'ASSURE': 2.0,
 'ASSURED': 2.0,
 'ASSURES': 2.0,
 'ASSURING': 2.0,
 'ATTAIN': 2.0,
 'ATTAINED': 2.0,
 'ATTAINING': 2.0,
 'ATTAINMENT': 2.0,
 'ATTAINMENTS': 2.0,
 'ATTAINS': 2.0,
 'ATTRACTIVE': 2.0,
 'ATTRACTIVENESS': 2.0,
 'BEAUTIFUL': 2.0,
 'BEAUTIFULLY': 2.0,
 'BENEFICIAL': 2.0,
 'BENEFICIALLY': 2.0,
 'BENEFIT': 2.0,
 'BENEFITED': 2.0,
 'BENEFITING': 2.0,
 'BENEFITTED': 2.0,
 'BENEFITTING': 2.0,
 'BEST': 2.0,
 'BETTER': 2.0,
 'BOL

In [ ]:
sentiment=[]
for tweet in ftext_tweets['tweet_text']:
    sentiment.append(sia.polarity_scores(tweet)['compound'])
ftext_tweets['Sentiment Score']=sentiment

In [ ]:
pd.set_option('display.max_colwidth', -1)
ftext_tweets[ftext_tweets['tweet_text'].str.len()<150].sample(10)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,tweet_text,Sentiment Score
date,,
2019-07-23 17:35:00,@SaraCarterDC @RashidaTlaib I don't think she understands how the economy works...,-0.2670
2019-07-01 20:43:00,🎧 Hustlaz Ambition by @YoungJeezy on @PandoraMusic https://t.co/79pmz23vom I guess he ain’ lame enough..,-0.2250
2019-08-19 18:14:00,@rebeccaballhaus @maggieNYT What recession?,-0.4010
2019-08-27 14:55:00,@cd_hooks I entered boot camp 6 months before 9/11 and left the military 6 months before the recession. I’m not great at this.,0.3182
2019-07-01 21:02:00,“The stock market is kinda like a rich people’s board game”,0.7973
2019-06-24 21:18:00,Loans didn’t get us out of the recession it was those randomers who sold strawberries on the middle of the road,-0.2665
2019-08-26 15:25:00,My stock goin crazy pre market wtf 😦,-0.6571
2019-08-14 19:25:00,"Recession 1 year out , tops 2. Called it, can look at this post in future...",0.2389
2019-07-26 16:09:00,Inflation out now\nFull video in comments https://t.co/rVWQ7UTrdb,0.3423


In [ ]:
# Downsample the tweet sentiment scores
iss = ftext_tweets['Sentiment Score'].resample(freq,base=15.5,label='right').mean().to_frame()
iss = iss[iss['Sentiment Score'].notnull()]
iss['Z Scores']=(iss['Sentiment Score']-iss['Sentiment Score'].mean())/iss['Sentiment Score'].std()

In [ ]:
iss

,Sentiment Score,Z Scores
date,,
2019-06-24 20:30:00,0.063693,0.105398
2019-06-24 21:30:00,0.132368,0.803240
2019-06-25 15:30:00,0.172388,1.209906
2019-06-25 16:30:00,0.198150,1.471683
2019-06-25 17:30:00,0.132584,0.805440
...,...,...
2019-09-03 17:30:00,-0.055798,-1.108808
2019-09-03 18:30:00,-0.010625,-0.649777
2019-09-03 19:30:00,0.028067,-0.256610


In [ ]:
ftext_tweets

,tweet_text,Sentiment Score
date,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379
...,...,...
2019-09-03 20:47:00,"@PrimeXbt In general, the SP500 is clearly overrated https://t.co/YOzStZn4Gu",0.4051
2019-09-03 20:47:00,@AmandaAgatiPNC @NFIB Are you a recession denier? The bond market is screaming recession. I don't think it is different this time...,-0.8789
2019-09-03 20:47:00,"@pdtugs @CPC_HQ Actually Canada economy is good, American is headed to a recession thanks to Trump. Once again, good stewards of the economy, Liberals.",0.5850


make table with three fields: Date (day), volume of tweets, bullish percentage

In [ ]:
cleaned_tweets = ftext_tweets

In [ ]:
# clean_tweets = ftext_tweets["date"] = 
# filter into days
days = []

for x in range(cleaned_tweets.count()[0]):
    days.append(str(cleaned_tweets.index[x])[:10])
cleaned_tweets['day'] = days
cleaned_tweets.head()

,tweet_text,Sentiment Score,day
date,,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400,2019-06-24
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282,2019-06-24
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321,2019-06-24
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587,2019-06-24
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379,2019-06-24


In [ ]:
# get average sentiment score for each day
x = cleaned_tweets.groupby('day', as_index=False)['Sentiment Score'].mean()
# get total number of tweets for each day
y = cleaned_tweets.groupby('day', as_index=False)['tweet_text'].count()
# percentage bullish


In [ ]:
# percentage bullish
z = cleaned_tweets.groupby('day', as_index=False)['sentiment_boolean'].sum() # number of bullish tweets on the day
z_list = list(z['sentiment_boolean'])
y_list = list(y['tweet_text'])

In [ ]:
sentiment_boolean = []
for y in range(cleaned_tweets.count()[0]):
    if cleaned_tweets['Sentiment Score'][y] >= 0:
        sentiment_boolean.append(1)
    else:
        sentiment_boolean.append(0)
cleaned_tweets['sentiment_boolean'] = sentiment_boolean
cleaned_tweets.head()

,tweet_text,Sentiment Score,day,sentiment_boolean
date,,,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400,2019-06-24,0
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282,2019-06-24,1
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321,2019-06-24,1
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587,2019-06-24,0
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379,2019-06-24,0


In [ ]:
z_list[0]/y_list[0]

0.5625

In [ ]:
z_percent = []
for i in range(len(z_list)):
    z_percent.append(z_list[i]/y_list[i])
z_percent

[0.5625,
 0.6272787173292335,
 0.622778382299601,
 0.6036036036036037,
 0.665938864628821,
 0.6409415971394518,
 0.6696546269443712,
 0.6134146341463415,
 0.5715919487648673,
 0.6070469798657718,
 0.5740861618798956,
 0.5643111337061515,
 0.5536919221129748,
 0.6349892008639308,
 0.6074332171893148,
 0.5831024930747922,
 0.5983115468409586,
 0.5733678086619263,
 0.5982982534706672,
 0.5803033951360462,
 0.6181520850367948,
 0.6318021201413427,
 0.5574288724973656,
 0.5560514570753479,
 0.5658279695728496,
 0.5724310776942356,
 0.5221206581352834,
 0.5589340621797062,
 0.5428015564202334,
 0.5361503856041131,
 0.540333660451423,
 0.5231900452488688,
 0.5363272983776158,
 0.48408746481922493,
 0.49032258064516127,
 0.5435852881552273,
 0.3734301709789681,
 0.4145185185185185,
 0.4377851769638673,
 0.38443766749838276,
 0.40302013422818794,
 0.42262501138537206,
 0.4492753623188406,
 0.47636717024472125,
 0.5093916939514045,
 0.4559244901398955,
 0.48754318967084925,
 0.4980671414038657,


In [ ]:
tweets_summary = x

In [ ]:
tweets_summary['Volume'] = y_list
tweets_summary['BullishDecimal'] = z_percent
tweets_summary['TotalBullish'] = z_list

In [ ]:
# tweets_summary.columns = ['Day', 'Average Sentiment Score', 'Volume', 'BullishDecimal', 'TotalBullish']
# sum(list(tweets_summary['Volume']))
# tweets_summary.index.tolist()

In [ ]:
tweets_summary.head()

,day,Sentiment Score,Volume,BullishDecimal,TotalBullish
0,2019-06-24,0.112120,736,0.562500,414
1,2019-06-25,0.162006,4553,0.627279,2856
2,2019-06-26,0.148589,2757,0.622778,1717
3,2019-06-27,0.127418,2664,0.603604,1608
4,2019-06-28,0.204582,4122,0.665939,2745


In [ ]:
cleaned_prices = prices
days_p = []
times_p = []
for j in range(prices.count()[0]):
    days_p.append(str(cleaned_prices.index[j])[:10])
    times_p.append(str(cleaned_prices.index[j])[11:])
cleaned_prices['day'] = days_p
cleaned_prices['times'] = times_p

In [ ]:
cleaned_prices.tail()

,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log,day,times
dates,,,,,,,,,,
2019-09-27 17:30:00,26890.71,2970.09,-0.004136,-0.004612,0.438163,0.294099,10.199536,7.996348,2019-09-27,17:30:00
2019-09-27 18:30:00,26825.86,2961.04,-0.002415,-0.003052,0.309801,0.111971,10.197122,7.993296,2019-09-27,18:30:00
2019-09-27 19:30:00,26776.07,2954.41,-0.001858,-0.002242,0.211249,-0.021455,10.195264,7.991054,2019-09-27,19:30:00
2019-09-27 20:30:00,26755.14,2952.05,-0.000782,-0.000799,0.169821,-0.068950,10.194482,7.990255,2019-09-27,20:30:00
2019-09-27 21:30:00,26820.25,2961.79,0.002431,0.003294,0.298697,0.127064,10.196912,7.993549,2019-09-27,21:30:00


In [ ]:
max_time = cleaned_prices['times'].max()
# Get names of indexes for which column times is not max time
non_closing_times = cleaned_prices[cleaned_prices['times'] != max_time ].index
# Delete these row indexes from dataFrame
cleaned_prices.drop(non_closing_times , inplace=True)

In [ ]:
# only keep days that match up with days from tweets_summary df
cleaned_prices.head()

,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log,day,times
dates,,,,,,,,,,
2019-06-21 21:30:00,26719.13,2950.46,-0.002766,-0.001676,0.098545,-0.100948,10.193135,7.989716,2019-06-21,21:30:00
2019-06-24 21:30:00,26727.54,2945.35,-0.000733,-0.000597,0.115191,-0.203785,10.193450,7.987983,2019-06-24,21:30:00
2019-06-25 21:30:00,26548.22,2917.38,-0.000915,-0.001740,-0.239747,-0.766671,10.186718,7.978441,2019-06-25,21:30:00
2019-06-26 21:30:00,26536.82,2913.78,-0.001711,-0.000799,-0.262312,-0.839120,10.186288,7.977206,2019-06-26,21:30:00
2019-06-27 21:30:00,26526.58,2924.92,-0.001277,-0.000243,-0.282580,-0.614931,10.185903,7.981022,2019-06-27,21:30:00


In [ ]:
tweets_summary = tweets_summary.set_index('day')
cleaned_prices = cleaned_prices.set_index('day')

In [ ]:
days_column = tweets_summary.index.tolist()
days_column.remove('2019-07-03')
len(days_column)

49

In [ ]:
result = pd.concat([cleaned_prices, tweets_summary], axis=1, join='inner')
result['day'] = days_column
result = result.reset_index(drop=True)
result.rename(columns={'day': 'Day'}, inplace=True)
result.to_csv(r"C:\Users\Owner\Desktop\Projects\stock_data.csv", header=True)
result

,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log,times,Sentiment Score,Volume,BullishDecimal,TotalBullish,Day
0,26727.54,2945.35,-0.000733,-0.000597,0.115191,-0.203785,10.193450,7.987983,21:30:00,0.112120,736,0.562500,414,2019-06-24
1,26548.22,2917.38,-0.000915,-0.001740,-0.239747,-0.766671,10.186718,7.978441,21:30:00,0.162006,4553,0.627279,2856,2019-06-25
2,26536.82,2913.78,-0.001711,-0.000799,-0.262312,-0.839120,10.186288,7.977206,21:30:00,0.148589,2757,0.622778,1717,2019-06-26
3,26526.58,2924.92,-0.001277,-0.000243,-0.282580,-0.614931,10.185903,7.981022,21:30:00,0.127418,2664,0.603604,1608,2019-06-27
4,26599.96,2941.76,0.002416,0.003374,-0.137335,-0.276032,10.188665,7.986763,21:30:00,0.204582,4122,0.665939,2745,2019-06-28
5,26717.43,2964.33,0.001007,0.001303,0.095180,0.178181,10.193071,7.994406,21:30:00,0.166332,3356,0.640942,2151,2019-07-01
6,26786.68,2973.01,0.002528,0.002290,0.232250,0.352863,10.195660,7.997330,21:30:00,0.167932,3793,0.669655,2540,2019-07-02
7,26922.12,2990.41,-0.000214,-0.000254,0.500334,0.703032,10.200704,8.003166,21:30:00,0.099155,4372,0.571592,2499,2019-07-05
8,26806.14,2975.95,-0.000030,0.000013,0.270768,0.412029,10.196386,7.998319,21:30:00,0.141956,2980,0.607047,1809,2019-07-08
9,26783.49,2979.63,0.002118,0.002046,0.225936,0.486088,10.195541,7.999554,21:30:00,0.105030,3064,0.574086,1759,2019-07-09


In [ ]:
#PIE CHART 
# posi=0
# nega=0
# for i in range (0,len(tweets_summary)):
#     get_val=tweets_summary['Average Sentiment Score'][i]
#     if(float(get_val)<(0)):
#         nega=nega+1
#     if(float(get_val>(0))):
#         posi=posi+1
# posper=(posi/(len(tweets_summary)))*100
# negper=(nega/(len(tweets_summary)))*100
# print("% of positive tweets= ",posper)
# print("% of negative tweets= ",negper)
# arr=np.asarray([posper,negper], dtype=int)
# plt.pie(arr,labels=['positive','negative'])
# plt.plot()

In [ ]:
#train_start_date = '2019-06-24'
#train_end_date = '2019-07-24'
#test_start_date = '2019-07-25'
#test_end_date = '2019-09-27'
train_start_index = '0'
train_end_index = '40'
test_start_index = '41'
test_end_index = '49'
#train = tweets_summary.loc[train_start_date : train_end_date]
#test = tweets_summary.loc[test_start_date:test_end_date]
train = result.loc[train_start_index : train_end_index]
test = result.loc[test_start_index:test_end_index]

In [ ]:
sentiment_score_list = []
for day, row in train.T.iteritems():
    sentiment_score = np.asarray([result.loc[day, 'Sentiment Score'],result.loc[day, 'Volume']])
    sentiment_score_list.append(sentiment_score)
numpy_df_train = np.asarray(sentiment_score_list)

In [ ]:
print(numpy_df_train)

[[ 1.12119701e-01  7.36000000e+02]
 [ 1.62005601e-01  4.55300000e+03]
 [ 1.48589300e-01  2.75700000e+03]
 [ 1.27417530e-01  2.66400000e+03]
 [ 2.04581951e-01  4.12200000e+03]
 [ 1.66331913e-01  3.35600000e+03]
 [ 1.67931795e-01  3.79300000e+03]
 [ 9.91554895e-02  4.37200000e+03]
 [ 1.41955772e-01  2.98000000e+03]
 [ 1.05030287e-01  3.06400000e+03]
 [ 8.77147941e-02  3.93400000e+03]
 [ 9.35252940e-02  5.18700000e+03]
 [ 1.60023419e-01  3.24100000e+03]
 [ 1.25249826e-01  3.44400000e+03]
 [ 1.06645346e-01  3.61000000e+03]
 [ 1.35793437e-01  3.67200000e+03]
 [ 1.07922409e-01  4.64100000e+03]
 [ 1.28654165e-01  4.46600000e+03]
 [ 1.18689477e-01  4.15300000e+03]
 [ 1.47888580e-01  3.66900000e+03]
 [ 1.65864099e-01  2.83000000e+03]
 [ 7.24528978e-02  3.79600000e+03]
 [ 7.44862431e-02  3.80900000e+03]
 [ 9.27133119e-02  3.41800000e+03]
 [ 8.34791980e-02  3.99000000e+03]
 [ 3.97744241e-02  5.47000000e+03]
 [ 7.47868466e-02  5.85400000e+03]
 [ 5.62448638e-02  5.14000000e+03]
 [ 3.03854756e-02  6

In [ ]:
sentiment_score_list = []
for day, row in test.T.iteritems():
    sentiment_score = np.asarray([result.loc[day, 'Sentiment Score'],result.loc[day, 'Volume']])
    sentiment_score_list.append(sentiment_score)
numpy_df_test = np.asarray(sentiment_score_list)

In [ ]:
print(numpy_df_test)

[[-7.32199275e-02  7.45200000e+03]
 [-3.91301175e-02  1.13190000e+04]
 [ 2.84785456e-03  5.80300000e+03]
 [-1.11055115e-02  5.93300000e+03]
 [-2.79378978e-02  5.49900000e+03]
 [-6.41182096e-03  4.91500000e+03]
 [-5.22804745e-02  6.57600000e+03]
 [-1.88148796e-03  5.14800000e+03]]


In [ ]:
y_train = pd.DataFrame(train['SP500 CLOSE'])
y_test = pd.DataFrame(test['SP500 CLOSE'])
# print(y_train)
# print(y_test)

## RANDOM FOREST REGRESSION PLOT, daily buckets, WITH ACCURACY SCORE, etc.

In [ ]:
from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
rf = RandomForestRegressor()
rf.fit(numpy_df_train, y_train)

# from sklearn.neural_network import MLPClassifier
# mlpc = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', #'relu', the rectified linear unit function
#                      solver='lbfgs', alpha=0.005, learning_rate_init = 0.001, shuffle=False)
# mlpc.fit(numpy_df_train, y_train['SP500 CLOSE'].tolist())
# prediction = mlpc.predict(numpy_df_test)

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


RandomForestRegressor()

In [ ]:
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

RandomForestRegressor(n_estimators=10, n_jobs=1, random_state=42)

Prediction

In [ ]:
prediction, bias, contributions = ti.predict(rf, numpy_df_test)
print(prediction)

[[2896.4552]
 [2909.4612]
 [2882.6075]
 [2883.0933]
 [2908.8674]
 [2890.3373]
 [2892.7794]
 [2886.2134]]


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
idx=np.arange(int(test_start_index),int(test_end_index))
predictions_df = pd.DataFrame(data=prediction[0:], index = idx, columns=['SP500 CLOSE'])
predictions_df.rename(columns={'SP500 CLOSE':'predicted_price'}, inplace=True)
# predictions_df

In [ ]:
n = np.size(test['SP500 CLOSE'])
error = []
for x in range(n):
    err = predictions_df['predicted_price'].tolist()[x] - test['SP500 CLOSE'].tolist()[x]
    error.append(err*err)
mean_squared_error = np.mean(error)
mean_squared_error

993.6029709487158

In [ ]:
def calculateAccuracy(interval, test, predict):
    if len(test) != len(predict):
        return "data must be the same length"
    n = len(test)
    score = 0
    for x in range(n):
        if predict[x] < test[x] + interval and predict[x] > test[x] - interval:
            score+= 100/n
    return score /100
#     return (f"{score}%")

In [ ]:
calculateAccuracy(40,test['SP500 CLOSE'].tolist(), predictions_df['predicted_price'].tolist())

0.875

In [ ]:
predicted_vs_actual = []
n = np.size(test['SP500 CLOSE'])
for x in range(n):
    new_row = []
    new_row.append(predictions_df['predicted_price'].tolist()[x])
    new_row.append(test['SP500 CLOSE'].tolist()[x])
    new_row.append(predictions_df['predicted_price'].tolist()[x] - test['SP500 CLOSE'].tolist()[x])
    predicted_vs_actual.append(new_row)

In [ ]:
predicted_vs_actual

[[2896.455199999997, 2922.95, -26.49480000000267],
 [2909.4611999999966, 2847.11, 62.351199999996425],
 [2882.6075000000005, 2878.38, 4.227500000000418],
 [2883.093300000001, 2869.16, 13.933300000001054],
 [2908.8674000000005, 2887.94, 20.92740000000049],
 [2890.3373000000015, 2924.58, -34.24269999999842],
 [2892.7793999999976, 2926.46, -33.680600000002414],
 [2886.213400000002, 2906.27, -20.05659999999807]]

In [ ]:
# calculate r squared
correlation_matrix = np.corrcoef(test['SP500 CLOSE'].tolist(), predictions_df['predicted_price'].tolist())
correlation_xy = correlation_matrix[0,1]
r_squared = round(correlation_xy**2., 5)
print(r_squared)

0.06078


In [ ]:
# Plot
plt.clf()
start_day = int(test_start_index)
predicted = "RF-daily Model prediction"
actual = "Actual SP500 Closing Price"
classification_accuracy = calculateAccuracy(40,test['SP500 CLOSE'].tolist(), predictions_df['predicted_price'].tolist())
# r_squared
# mean_squared_error
# plt.text(100, 100,f"r-squared: {r_squared}", horizontalalignment='center',
#      verticalalignment='center',
#      transform=ax.transAxes)
# plt.text(10, 3,f"classification accuracy: {classification_accuracy}", horizontalalignment='center',
#      verticalalignment='center',
#      transform=ax.transAxes)
# plt.text(10, 4,f"mean squared error: {mean_squared_error}", horizontalalignment='center',
#      verticalalignment='center',
#      transform=ax.transAxes)


predicted, = plt.plot(result.index.tolist()[int(test_start_index):], predictions_df['predicted_price'].tolist(), marker="+",color="blue", linewidth=1, label=predicted)
actual, = plt.plot(result.index.tolist()[int(test_start_index):], test['SP500 CLOSE'].tolist(), marker="*",color="green", linewidth=1, label=actual)
plt.legend(handles=[predicted, actual], loc="best")
# Create labels for the X and Y axis
plt.title(f'Predicted vs. actual SP500 Closing Price')
plt.xlabel("Days")
plt.ylabel("SP500 Closing Price")
plt.savefig("daily_rf_model_results.png")
# predictions_df['predicted_price'] = predictions_df['predicted_price'].apply(np.int64)
# test['SP500 CLOSE']=test['SP500 CLOSE'].apply(np.int64)
# print(accuracy_score(test['SP500 CLOSE'],predictions_df['predicted_price']))
# print(rf.score(numpy_dataframe_train, train['SP500 CLOSE']))

In [ ]:
# Create dataframe for random forest, daily
predicted = predictions_df['predicted_price'].tolist()
actual = test['SP500 CLOSE'].tolist()
squared_error = []
for x in range(len(predicted)):
    squared_error.append((predicted[x] - actual[x])*(predicted[x] - actual[x]))
dates = result['Day'].tolist()[int(test_start_index):]
daily_rf_df = pd.DataFrame({'Predicted SP500 Prices': predicted, 'Actual SP500 Prices': actual, 'Squared Error': squared_error, 'Date': dates})
daily_rf_df.to_csv(r"C:\Users\Owner\Desktop\Projects\daily_rf_df.csv", header=True)
daily_rf_df

,Predicted SP500 Prices,Actual SP500 Prices,Squared Error,Date
0,2896.4552,2922.95,701.974427,2019-08-22
1,2909.4612,2847.11,3887.672141,2019-08-23
2,2882.6075,2878.38,17.871756,2019-08-26
3,2883.0933,2869.16,194.136849,2019-08-27
4,2908.8674,2887.94,437.956071,2019-08-28
5,2890.3373,2924.58,1172.562503,2019-08-29
6,2892.7794,2926.46,1134.382816,2019-08-30
7,2886.2134,2906.27,402.267204,2019-09-03


In [ ]:
accuracy = []
x = 0
last = 0
while True:
# for x in range(51):
    acc = round(calculateAccuracy(x,test['SP500 CLOSE'].tolist(), predictions_df['predicted_price'].tolist()), 4)
    accuracy.append(acc)
    if acc >= 1:
        break
    x+=1
daily_rf_classification_df = pd.DataFrame({'Classification accuracy': accuracy})
daily_rf_classification_df.round({'Classification accuracy': 4})
daily_rf_classification_df.to_csv(r"C:\Users\Owner\Desktop\Projects\daily_rf_classification_df.csv", header=True)
daily_rf_classification_df

,Classification accuracy
0,0.000
1,0.000
2,0.000
3,0.000
4,0.000
...,...
59,0.875
60,0.875
61,0.875
62,0.875


## DAILY Decision tree

In [ ]:
dtr = DecisionTreeRegressor()
dtr.fit(numpy_df_train, y_train)

DecisionTreeRegressor()

In [ ]:
DecisionTreeRegressor(criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, random_state=42)

DecisionTreeRegressor(max_features='auto', random_state=42)

In [ ]:
prediction, bias, contributions = ti.predict(dtr, numpy_df_test)

In [ ]:
idx=np.arange(int(test_start_index),int(test_end_index))
predictions_df = pd.DataFrame(data=prediction[0:], index = idx, columns=['SP500 CLOSE'])
predictions_df.rename(columns={'SP500 CLOSE':'predicted_price'}, inplace=True)
predictions_df

,predicted_price
41,2888.68
42,2924.43
43,2883.98
44,2883.98
45,2918.65
46,2883.98
47,2888.68
48,2883.98


In [ ]:
# Create dataframe for decison tree, daily
predicted = predictions_df['predicted_price'].tolist()
actual = test['SP500 CLOSE'].tolist()
squared_error = []
for x in range(len(predicted)):
    squared_error.append((predicted[x] - actual[x])*(predicted[x] - actual[x]))
dates = result['Day'].tolist()[int(test_start_index):]
daily_dtr_df = pd.DataFrame({'Predicted SP500 Prices': predicted, 'Actual SP500 Prices': actual, 'Squared Error': squared_error, 'Date': dates})
daily_dtr_df.to_csv(r"C:\Users\Owner\Desktop\Projects\daily_dtr_df.csv", header=True)
daily_dtr_df

,Predicted SP500 Prices,Actual SP500 Prices,Squared Error,Date
0,2888.68,2922.95,1174.4329,2019-08-22
1,2924.43,2847.11,5978.3824,2019-08-23
2,2883.98,2878.38,31.3600,2019-08-26
3,2883.98,2869.16,219.6324,2019-08-27
4,2918.65,2887.94,943.1041,2019-08-28
5,2883.98,2924.58,1648.3600,2019-08-29
6,2888.68,2926.46,1427.3284,2019-08-30
7,2883.98,2906.27,496.8441,2019-09-03


In [ ]:
accuracy = []
x = 0
last = 0
while True:
# for x in range(51):
    acc = round(calculateAccuracy(x,test['SP500 CLOSE'].tolist(), predictions_df['predicted_price'].tolist()), 4)
    accuracy.append(acc)
    if acc >= 1:
        break
    x+=1
daily_dtr_classification_df = pd.DataFrame({'Classification accuracy': accuracy})
daily_dtr_classification_df.round({'Classification accuracy': 4})
daily_dtr_classification_df.to_csv(r"C:\Users\Owner\Desktop\Projects\daily_dtr_classification_df.csv", header=True)
daily_dtr_classification_df

,Classification accuracy
0,0.000
1,0.000
2,0.000
3,0.000
4,0.000
...,...
74,0.875
75,0.875
76,0.875
77,0.875


In [ ]:
# Plot
plt.clf()
start = int(test_start_index)
predicted = "DTR-daily Model prediction"
actual = "Actual SP500 Closing Price"

predicted, = plt.plot(result.index.tolist()[int(test_start_index):], predictions_df['predicted_price'].tolist(), marker="+",color="blue", linewidth=1, label=predicted)
actual, = plt.plot(result.index.tolist()[int(test_start_index):], test['SP500 CLOSE'].tolist(), marker="*",color="green", linewidth=1, label=actual)
plt.legend(handles=[predicted, actual], loc="best")
# Create labels for the X and Y axis
plt.title(f'Predicted vs. actual SP500 hourly Price')
plt.xlabel("Day")
plt.ylabel("SP500 Closing Price")
plt.savefig("dtr_daily_model_results.png")

## RUN THE ML AGAIN but this time aggregating tweets per hour to get more data

In [ ]:
# z = cleaned_tweets.groupby('date', as_index=True)['date'] # number of bullish tweets on the hour
ftext_tweets.head()

,tweet_text,Sentiment Score,day,sentiment_boolean
date,,,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400,2019-06-24,0
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282,2019-06-24,1
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321,2019-06-24,1
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587,2019-06-24,0
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379,2019-06-24,0


In [ ]:
# filter into hours
hours = []
my_tweets = ftext_tweets
for x in range(my_tweets.count()[0]):
    hours.append(str(my_tweets.index[x])[11:13])
my_tweets['hour'] = hours
my_tweets.head()

,tweet_text,Sentiment Score,day,sentiment_boolean,hour
date,,,,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400,2019-06-24,0,19
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282,2019-06-24,1,19
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321,2019-06-24,1,19
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587,2019-06-24,0,19
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379,2019-06-24,0,19


In [ ]:
# create new data column
hours_days = []
for x in range(my_tweets.count()[0]):
    day = my_tweets['day'][x]
    hour = my_tweets['hour'][x]
    result = day+"-"+hour
    hours_days.append(result)
my_tweets['day_hour'] = hours_days
my_tweets.head()

,tweet_text,Sentiment Score,day,sentiment_boolean,hour,day_hour
date,,,,,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400,2019-06-24,0,19,2019-06-24-19
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282,2019-06-24,1,19,2019-06-24-19
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321,2019-06-24,1,19,2019-06-24-19
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587,2019-06-24,0,19,2019-06-24-19
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379,2019-06-24,0,19,2019-06-24-19


In [ ]:
# get average sentiment score for each hour/day
x = my_tweets.groupby('day_hour', as_index=False)['Sentiment Score'].mean()
# get total number of tweets for each our/day
y = my_tweets.groupby('day_hour', as_index=False)['tweet_text'].count()

In [ ]:
hourly_tweets_summary = x 

In [ ]:
hourly_tweets_summary["Volume"] = y['tweet_text'].tolist()

In [ ]:
hourly_tweets_summary.head()

,day_hour,Sentiment Score,Volume
0,2019-06-24-19,0.139900,9
1,2019-06-24-20,0.096132,476
2,2019-06-24-21,0.141443,251
3,2019-06-25-14,0.157871,425
4,2019-06-25-15,0.200516,754


In [ ]:
# reading in stock index date
prices=pd.read_csv("prices_60m.csv",parse_dates=['Dates'])
prices.rename(columns={'Dates':'dates'},inplace=True)
prices=prices.set_index('dates')
prices=prices.dropna(how='any')
prices

if missing_data_handling:
    prices=prices.resample(freq,base=15.5).asfreq()
    
# calculating logreturns and Z Scores
def logreturn(x):
    return np.log(x/x.shift(1))

def zscore(x):
    return (x-x.mean())/x.std()

prices['DJIA LogRet']=logreturn(prices['DJIA CLOSE'])
prices['SP500 LogRet']=logreturn(prices['SP500 CLOSE'])
prices['DJIA Z Scores']=zscore(prices['DJIA CLOSE'])
prices['SP500 Z Scores']=zscore(prices['SP500 CLOSE'])
prices['DJIA Log']=np.log(prices['DJIA CLOSE'])
prices['SP500 Log']=np.log(prices['SP500 CLOSE'])
prices=prices.dropna(how='any')
prices

,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log
dates,,,,,,,,
2019-06-21 17:30:00,26791.51,2952.75,-0.001901,-0.001739,0.241810,-0.054862,10.195840,7.990492
2019-06-21 18:30:00,26796.05,2953.07,0.000169,0.000108,0.250797,-0.048422,10.196010,7.990601
2019-06-21 19:30:00,26790.67,2953.51,-0.000201,0.000149,0.240148,-0.039568,10.195809,7.990750
2019-06-21 20:30:00,26793.13,2955.41,0.000092,0.000643,0.245017,-0.001331,10.195901,7.991393
2019-06-21 21:30:00,26719.13,2950.46,-0.002766,-0.001676,0.098545,-0.100948,10.193135,7.989716
...,...,...,...,...,...,...,...,...
2019-09-27 17:30:00,26890.71,2970.09,-0.004136,-0.004612,0.438163,0.294099,10.199536,7.996348
2019-09-27 18:30:00,26825.86,2961.04,-0.002415,-0.003052,0.309801,0.111971,10.197122,7.993296
2019-09-27 19:30:00,26776.07,2954.41,-0.001858,-0.002242,0.211249,-0.021455,10.195264,7.991054


In [ ]:
hourly_cleaned_prices = prices
days_p = []
hours_p = []
day_time_p = []
for j in range(hourly_cleaned_prices.count()[0]):
    days_p.append(str(hourly_cleaned_prices.index[j])[:10])
    hours_p.append(str(hourly_cleaned_prices.index[j])[11:13])
    day_time_p.append(str(hourly_cleaned_prices.index[j])[:10]+"-"+str(hourly_cleaned_prices.index[j])[11:13])
hourly_cleaned_prices['day'] = days_p
hourly_cleaned_prices['hour'] = hours_p
hourly_cleaned_prices['day_hour'] = day_time_p

In [ ]:
hourly_tweets_summary = hourly_tweets_summary.set_index('day_hour')
hourly_cleaned_prices = hourly_cleaned_prices.set_index('day_hour')

In [ ]:
day_hour_column = hourly_tweets_summary.index.tolist()
len(day_hour_column)

388

In [ ]:
hourly_result = pd.concat([hourly_cleaned_prices, hourly_tweets_summary], axis=1, join='inner')
# hourly_result['day_hour'] = days_column
hourly_result = hourly_result.reset_index(drop=False)
# hourly_result.rename(columns={'day': 'Day'}, inplace=True)
# hourly_result.to_csv(r"C:\Users\Owner\Desktop\Projects\hourly_stock_data.csv", header=True)
hourly_result

,day_hour,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log,day,hour,Sentiment Score,Volume
0,2019-06-24-19,26735.66,2947.36,-0.000659,-0.000885,0.131263,-0.163334,10.193754,7.988665,2019-06-24,19,0.139900,9
1,2019-06-24-20,26747.13,2947.11,0.000429,-0.000085,0.153967,-0.168365,10.194182,7.988580,2019-06-24,20,0.096132,476
2,2019-06-24-21,26727.54,2945.35,-0.000733,-0.000597,0.115191,-0.203785,10.193450,7.987983,2019-06-24,21,0.141443,251
3,2019-06-25-15,26648.87,2937.03,-0.002948,-0.002829,-0.040525,-0.371222,10.190502,7.985154,2019-06-25,15,0.200516,754
4,2019-06-25-16,26685.95,2935.24,0.001390,-0.000610,0.032870,-0.407245,10.191892,7.984545,2019-06-25,16,0.169478,715
...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,2019-09-03-16,26061.46,2902.58,0.000373,0.000572,-1.203218,-1.064516,10.168213,7.973355,2019-09-03,16,-0.081800,1040
334,2019-09-03-17,26067.01,2903.98,0.000213,0.000482,-1.192232,-1.036341,10.168426,7.973837,2019-09-03,17,-0.012143,790
335,2019-09-03-18,26048.81,2900.85,-0.000698,-0.001078,-1.228257,-1.099331,10.167727,7.972759,2019-09-03,18,0.004418,829
336,2019-09-03-19,26050.25,2898.69,0.000055,-0.000745,-1.225406,-1.142801,10.167783,7.972014,2019-09-03,19,0.034896,731


In [ ]:
train_start_index = 0
train_end_index = round(hourly_result.count()[0]*.8,0)
test_start_index = train_end_index+1
test_end_index = hourly_result.count()[0]
#train = tweets_summary.loc[train_start_date : train_end_date]
#test = tweets_summary.loc[test_start_date:test_end_date]
train = hourly_result.loc[train_start_index : train_end_index]
test = hourly_result.loc[test_start_index:test_end_index]

In [ ]:
sentiment_score_list = []
for hour, row in train.T.iteritems():
    sentiment_score = np.asarray([hourly_result.loc[hour, 'Sentiment Score'],hourly_result.loc[hour, 'Volume']])
    sentiment_score_list.append(sentiment_score)
numpy_df_train = np.asarray(sentiment_score_list)

In [ ]:
sentiment_score_list = []
for hour, row in test.T.iteritems():
    sentiment_score = np.asarray([hourly_result.loc[hour, 'Sentiment Score'],hourly_result.loc[hour, 'Volume']])
    sentiment_score_list.append(sentiment_score)
numpy_df_test = np.asarray(sentiment_score_list)

In [ ]:
y_train = pd.DataFrame(train['SP500 CLOSE'])
y_test = pd.DataFrame(test['SP500 CLOSE'])

In [ ]:
rf = RandomForestRegressor()
rf.fit(numpy_df_train, y_train)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor()

In [ ]:
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

RandomForestRegressor(n_estimators=10, n_jobs=1, random_state=42)

In [ ]:
prediction, bias, contributions = ti.predict(rf, numpy_df_test)

In [ ]:
idx=np.arange(int(test_start_index),int(test_end_index))
predictions_df = pd.DataFrame(data=prediction[0:], index = idx, columns=['SP500 CLOSE'])
predictions_df.rename(columns={'SP500 CLOSE':'predicted_price'}, inplace=True)

In [ ]:
n = np.size(test['SP500 CLOSE'])
error = []
for x in range(n):
    err = predictions_df['predicted_price'].tolist()[x] - test['SP500 CLOSE'].tolist()[x]
    error.append(err*err)
mean_squared_error = np.mean(error)
mean_squared_error

1035.0409095638734

In [ ]:
calculateAccuracy(40,test['SP500 CLOSE'].tolist(), predictions_df['predicted_price'].tolist())

0.8208955223880592

In [ ]:
predicted_vs_actual = []
n = np.size(test['SP500 CLOSE'])
for x in range(n):
    new_row = []
    new_row.append(predictions_df['predicted_price'].tolist()[x])
    new_row.append(test['SP500 CLOSE'].tolist()[x])
    new_row.append(predictions_df['predicted_price'].tolist()[x] - test['SP500 CLOSE'].tolist()[x])
    predicted_vs_actual.append(new_row)

In [ ]:
predicted_vs_actual

[[2899.7923999999994, 2918.32, -18.527600000000803],
 [2894.908800000002, 2915.97, -21.061199999997825],
 [2880.7139000000006, 2907.98, -27.26609999999937],
 [2928.8884999999987, 2909.64, 19.248499999998785],
 [2901.670599999999, 2900.51, 1.1605999999987944],
 [2845.3484, 2923.89, -78.54160000000002],
 [2845.3179999999998, 2925.37, -80.05200000000013],
 [2854.0496000000007, 2924.88, -70.83039999999937],
 [2907.922500000002, 2925.68, -17.75749999999789],
 [2942.2468, 2925.75, 16.496799999999894],
 [2909.9915000000037, 2922.64, -12.648499999996147],
 [2914.4507, 2924.43, -9.979299999999967],
 [2903.7231000000033, 2921.41, -17.686899999996513],
 [2894.112200000002, 2913.31, -19.197799999998097],
 [2909.8474000000024, 2919.86, -10.012599999997747],
 [2902.4930000000036, 2926.1, -23.606999999996333],
 [2919.952499999998, 2923.48, -3.5275000000019645],
 [2886.272399999999, 2925.07, -38.79760000000124],
 [2894.004099999997, 2922.95, -28.945900000002894],
 [2893.633000000002, 2914.89, -21.2569

In [ ]:
# calculate r squared
correlation_matrix = np.corrcoef(test['SP500 CLOSE'].tolist(), predictions_df['predicted_price'].tolist())
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2.
print(r_squared)

0.10414173573321504


In [ ]:
# Plot
plt.clf()
start_day = int(test_start_index)
predicted = "RF-hourly Model prediction"
actual = "Actual SP500 Closing Price"
classification_accuracy = calculateAccuracy(40,test['SP500 CLOSE'].tolist(), predictions_df['predicted_price'].tolist())
# r_squared
# mean_squared_error
# plt.text(100, 100,f"r-squared: {r_squared}", horizontalalignment='center',
#      verticalalignment='center',
#      transform=ax.transAxes)
# plt.text(10, 3,f"classification accuracy: {classification_accuracy}", horizontalalignment='center',
#      verticalalignment='center',
#      transform=ax.transAxes)
# plt.text(10, 4,f"mean squared error: {mean_squared_error}", horizontalalignment='center',
#      verticalalignment='center',
#      transform=ax.transAxes)


predicted, = plt.plot(hourly_result.index.tolist()[int(test_start_index):], predictions_df['predicted_price'].tolist(), marker="+",color="blue", linewidth=1, label=predicted)
actual, = plt.plot(hourly_result.index.tolist()[int(test_start_index):], test['SP500 CLOSE'].tolist(), marker="*",color="green", linewidth=1, label=actual)
plt.legend(handles=[predicted, actual], loc="best")
# Create labels for the X and Y axis
plt.title(f'Predicted vs. actual SP500 hourly Price')
plt.xlabel("hour&day")
plt.ylabel("SP500 Closing Price")
plt.savefig("rf_hourly_model_results.png")
# predictions_df['predicted_price'] = predictions_df['predicted_price'].apply(np.int64)
# test['SP500 CLOSE']=test['SP500 CLOSE'].apply(np.int64)
# print(accuracy_score(test['SP500 CLOSE'],predictions_df['predicted_price']))
# print(rf.score(numpy_dataframe_train, train['SP500 CLOSE']))

In [ ]:
hourly_result.head()

,day_hour,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log,day,hour,Sentiment Score,Volume
0,2019-06-24-19,26735.66,2947.36,-0.000659,-0.000885,0.131263,-0.163334,10.193754,7.988665,2019-06-24,19,0.139900,9
1,2019-06-24-20,26747.13,2947.11,0.000429,-0.000085,0.153967,-0.168365,10.194182,7.988580,2019-06-24,20,0.096132,476
2,2019-06-24-21,26727.54,2945.35,-0.000733,-0.000597,0.115191,-0.203785,10.193450,7.987983,2019-06-24,21,0.141443,251
3,2019-06-25-15,26648.87,2937.03,-0.002948,-0.002829,-0.040525,-0.371222,10.190502,7.985154,2019-06-25,15,0.200516,754
4,2019-06-25-16,26685.95,2935.24,0.001390,-0.000610,0.032870,-0.407245,10.191892,7.984545,2019-06-25,16,0.169478,715


In [ ]:
# Create dataframe for random forest, hourly
predicted = predictions_df['predicted_price'].tolist()
actual = test['SP500 CLOSE'].tolist()
squared_error = []
for x in range(len(predicted)):
    squared_error.append((predicted[x] - actual[x])*(predicted[x] - actual[x]))
times = hourly_result['day_hour'].tolist()[int(test_start_index):]
hourly_rf_df = pd.DataFrame({'Predicted SP500 Prices': predicted, 'Actual SP500 Prices': actual, 'Squared Error': squared_error, 'Time': times})
hourly_rf_df.to_csv(r"C:\Users\Owner\Desktop\Projects\hourly_rf_df.csv", header=True)
hourly_rf_df

,Predicted SP500 Prices,Actual SP500 Prices,Squared Error,Time
0,2899.7924,2918.32,343.271962,2019-08-20-17
1,2894.9088,2915.97,443.574145,2019-08-20-18
2,2880.7139,2907.98,743.440209,2019-08-20-19
3,2928.8885,2909.64,370.504752,2019-08-20-20
4,2901.6706,2900.51,1.346992,2019-08-20-21
...,...,...,...,...
62,2882.1866,2902.58,415.890764,2019-09-03-16
63,2922.5121,2903.98,343.438730,2019-09-03-17
64,2860.2017,2900.85,1652.284293,2019-09-03-18
65,2881.9040,2898.69,281.769796,2019-09-03-19


In [ ]:
accuracy = []
x = 0
last = 0
while True:
# for x in range(51):
    acc = round(calculateAccuracy(x,test['SP500 CLOSE'].tolist(), predictions_df['predicted_price'].tolist()), 4)
    accuracy.append(acc)
    if acc >= 1:
        break
    x+=1
hourly_rf_classification_df = pd.DataFrame({'Classification accuracy': accuracy})
hourly_rf_classification_df.round({'Classification accuracy': 4})
hourly_rf_classification_df.to_csv(r"C:\Users\Owner\Desktop\Projects\hourly_rf_classification_df.csv", header=True)
hourly_rf_classification_df

,Classification accuracy
0,0.0000
1,0.0000
2,0.0448
3,0.0746
4,0.1343
...,...
77,0.9552
78,0.9552
79,0.9851
80,0.9851


## HOURLY Decision Tree

In [ ]:
dtr = DecisionTreeRegressor()
dtr.fit(numpy_df_train, y_train)

DecisionTreeRegressor()

In [ ]:
DecisionTreeRegressor(criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, random_state=42)

DecisionTreeRegressor(max_features='auto', random_state=42)

In [ ]:
prediction, bias, contributions = ti.predict(dtr, numpy_df_test)

In [ ]:
idx=np.arange(int(test_start_index),int(test_end_index))
predictions_df = pd.DataFrame(data=prediction[0:], index = idx, columns=['SP500 CLOSE'])
predictions_df.rename(columns={'SP500 CLOSE':'predicted_price'}, inplace=True)
predictions_df

,predicted_price
271,2918.93
272,2890.95
273,2887.05
274,2939.75
275,2923.65
...,...
333,2887.05
334,2902.07
335,2864.26
336,2854.26


In [ ]:
# Create dataframe for decison tree, hourly
predicted = predictions_df['predicted_price'].tolist()
actual = test['SP500 CLOSE'].tolist()
squared_error = []
for x in range(len(predicted)):
    squared_error.append((predicted[x] - actual[x])*(predicted[x] - actual[x]))
times = hourly_result['day_hour'].tolist()[int(test_start_index):]
hourly_dtr_df = pd.DataFrame({'Predicted SP500 Prices': predicted, 'Actual SP500 Prices': actual, 'Squared Error': squared_error, 'Time': d})
hourly_dtr_df.to_csv(r"C:\Users\Owner\Desktop\Projects\hourly_dtr_df.csv", header=True)
hourly_dtr_df

TypeError: string indices must be integers

In [ ]:
accuracy = []
x = 0
last = 0
while True:
# for x in range(51):
    acc = round(calculateAccuracy(x,test['SP500 CLOSE'].tolist(), predictions_df['predicted_price'].tolist()), 4)
    accuracy.append(acc)
    if acc >= 1:
        break
    x+=1
hourly_dtr_classification_df = pd.DataFrame({'Classification accuracy': accuracy})
hourly_dtr_classification_df.round({'Classification accuracy': 4})
hourly_dtr_classification_df.to_csv(r"C:\Users\Owner\Desktop\Projects\hourly_dtr_classification_df.csv", header=True)
hourly_dtr_classification_df

In [ ]:
# Plot
plt.clf()
start = int(test_start_index)
predicted = "DTR-hourly Model prediction"
actual = "Actual SP500 Closing Price"

predicted, = plt.plot(hourly_result.index.tolist()[int(test_start_index):], predictions_df['predicted_price'].tolist(), marker="+",color="blue", linewidth=1, label=predicted)
actual, = plt.plot(hourly_result.index.tolist()[int(test_start_index):], test['SP500 CLOSE'].tolist(), marker="*",color="green", linewidth=1, label=actual)
plt.legend(handles=[predicted, actual], loc="best")
# Create labels for the X and Y axis
plt.title(f'Predicted vs. actual SP500 hourly Price')
plt.xlabel("hour&day")
plt.ylabel("SP500 Closing Price")
plt.savefig("dtr_hourly_model_results.png")